In [7]:
def extract_substring(text):
    # 找到 '@@\n' 的位置
    start_index = text.find("@@\n")
    
    # 如果找到了 '@@\n'，调整 start_index 跳过这三个字符
    if start_index != -1:
        start_index += 3  # 跳过 '@@' 和后面的换行符 '\n'
    else:
        return "未找到 '@@\\n'"

    # 找到第一个 '\n-' 的位置
    end_index = text.find("\n-")
    
    # 检查是否找到了 '\n-'
    if end_index == -1:
        return "未找到 '\\n-'"

    # 截取并返回中间的子字符串
    return text[start_index:end_index]


In [8]:
import re
import json

with open("/Users/maike/Desktop/fastjson/parser.go", 'r') as file:
    lines = file.readlines()
    
with open('/Users/maike/Desktop/fastjson/report.json', 'r') as file:
    # 加载 JSON 数据
    data = json.load(file)

function_pattern = re.compile(r'^func(?:\s+\(\s*\w+\s+\*?\w+\s*\))?(\s+\w+)')

l =[]
for killed in data["killed"]:
    # 使用正则表达式查找 - 到 , 之间的数字
    match = re.search(r'-([0-9]+),', killed["diff"])

    # 如果找到匹配项，则输出
    number = int(match.group(1)) # 提取第一个捕获组的内容
    
    sub = extract_substring(killed["diff"])
    number = number + sub.count('\n') + 1

    flag = False
    # 从指定行号开始向上搜索
    for i in range(number - 1, -1, -1):  # line_number - 1 因为行号通常是从1开始，而列表索引从0开始
        match = function_pattern.search(lines[i])
        if match:
            function_name = match.group(1).strip()  # 获取函数名称
            l.append(function_name)
            flag = True
            break
        
    if not flag:
        print(killed["processOutput"])

if len(l) != len(data["killed"]):
    print("errorrrrrrrrrr")
    
with open("function_names.json","w") as f:
    json.dump(l,f)

92
97
323
389
400
415
417
423
424
427
428
428
428
430
436
437
437
437
453
455
455
485
506
510
599
329
331
333
335
337
339
341
343
345
380
589
594
596
606
608
610
612
614
616
663
665
667
669
671
673
675
680
94
479
508
725
64
68
92
110
114
122
130
132
137
139
144
146
154
155
160
161
166
167
169
181
192
196
212
218
221
225
235
239
255
263
275
278
282
303
318
346
353
359
366
385
399
405
413
416
428
434
437
449
452
453
455
477
486
506
515
537
550
562
600
687
714
716
719
721
741
756
769
782
846
859
871
883
895
919
971
974
67
91
101
112
154
160
166
302
305
315
324
384
397
412
424
433
446
448
448
452
505
101
109
109
109
109
109
109
113
113
113
113
154
160
166
166
168
254
262
302
415
415
424
424
436
436
448
448
448
448
448
448
448
448
448
448
448
448
451
451
451
451
451
454
454
720
720
720
720
740
740
755
768
768
781
845
845
858
222
279
355
360
368
386
449
112
324
397
424
446
474
517
547
566
596
711
59
67
87
92
97
101
101
109
109
109
109
121
129
130
136
137
143
144
153
159
165
168
171
172
191
1